<a href="https://colab.research.google.com/github/Pathairush/ATM_optimization/blob/main/ipynb/atm_replenishment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp -q

In [ ]:
import numpy as np
import pandas as pd
import pulp
from decimal import *

# Implementation

To implement the LP solution for ATM replenishment process, we use the procedure from [[1]](https://www.igi-global.com/gateway/article/251842) reference as a main implementation.

Terminology
1. ***demand_day*** the day in which the demand amount is needed based on the forecasting methods.
2. ***holding_day*** the number of day that the demand amount has been hold for.
3. ***loading_day*** the day in which the demand amount of money is loaded into the ATM.

In [ ]:
# initial parameters
number_of_days = 5 # period of replenishment
cash_interest_rate = Decimal(1) / Decimal(100) # 1% per day
loading_cost = 5 # loading cost per times
demand_amount_per_day = [Decimal(e) for e in [100,200,100,300,100]] # amount to replenish from demand forecasting

In [ ]:
# calculate an amount with interest table
amount_df = pd.DataFrame(demand_amount_per_day, columns=['amount'])
amount_df['demand_day'] = [e + 1 for e in amount_df.index.to_list()]
amount_df['key'] = 0

print("Amount by each demand day")
display(amount_df.set_index(['demand_day'])[['amount']].T)

# create a period table
period_table = pd.DataFrame([Decimal(e) for e in range(number_of_days)], columns = ['holding_day'])
period_table['key'] = 0

# cross join the amount and period tables
amount_df = amount_df.merge(period_table, on='key', how='outer')
amount_df.drop(['key'], axis=1, inplace=True)
amount_df = amount_df[
                      (amount_df['demand_day'] > amount_df['holding_day'] )
]
amount_df['amount_with_interest'] = amount_df['amount'] * (1 + cash_interest_rate)**(amount_df['holding_day'])
amount_df['interest_cost'] = amount_df['amount_with_interest'] - amount_df['amount']

# calculate accumulated interests at each loading day
amount_df['loading_day'] = (amount_df['demand_day'] - amount_df['holding_day'])
amount_df = amount_df.sort_values(['loading_day','holding_day'])
amount_df['accumulated_cost'] = amount_df.groupby('loading_day')['interest_cost'].apply(np.cumsum)
amount_df['total_cost'] = amount_df['accumulated_cost'] + loading_cost

# shows results
print("\nInterest cost at each demand day if we loaded money at day 1")
display(pd.crosstab(index = amount_df['demand_day'],
                    columns = amount_df['holding_day'],
                    values= amount_df['interest_cost'], aggfunc='sum').fillna(0.0))

print("\nAccumulated interest cost at each loading day")
display(pd.crosstab(index = amount_df['loading_day'],
                    columns = amount_df['demand_day'],
                    values= amount_df['accumulated_cost'], aggfunc='sum').fillna(0.0))

print("\nTotal cost at each loading day")
total_costs = pd.crosstab(index = amount_df['loading_day'],
                    columns = amount_df['demand_day'],
                    values= amount_df['total_cost'], aggfunc='sum').fillna(0.0)
display(total_costs)

# convert decimal to float because pulp lib is not compatible with a decimal type.
for c in total_costs.columns:
    total_costs[c] = total_costs[c].astype(float)
total_costs_arr = np.array(total_costs)

Amount by each demand day


demand_day,1,2,3,4,5
amount,100,200,100,300,100



Interest cost at each demand day if we loaded money at day 1


holding_day,0,1,2,3,4
demand_day,,,,,
1,0,0,0,0,0
2,0,2.00,0,0,0
3,0,1.00,2.0100,0,0
4,0,3.00,6.0300,9.090300,0
5,0,1.00,2.0100,3.030100,4.06040100



Accumulated interest cost at each loading day


demand_day,1,2,3,4,5
loading_day,,,,,
1,0,2.00,4.0100,13.100300,17.16070100
2,0,0,1.00,7.0300,10.060100
3,0,0,0,3.00,5.0100
4,0,0,0,0,1.00
5,0,0,0,0,0



Total cost at each loading day


demand_day,1,2,3,4,5
loading_day,,,,,
1,5,7.00,9.0100,18.100300,22.16070100
2,0,5,6.00,12.0300,15.060100
3,0,0,5,8.00,10.0100
4,0,0,0,5,6.00
5,0,0,0,0,5


# Illustration of the LP solution on a case

In [ ]:
# an usage example
x = pulp.LpVariable("x", 0, 3)
y = pulp.LpVariable("y", 0, 1)
prob = pulp.LpProblem("myProblem", pulp.LpMinimize)
prob += x + y <= 2 # add a constraint
prob += -4 * x + y # add an objective
status = prob.solve()
print(f"The model state is {pulp.LpStatus[status]}")
print(f"The value of x is {pulp.value(x)}")

The model state is Optimal
The value of x is 2.0


## ATM Replenishment

In [ ]:
# create decision variables
variable_names = [str(i)+str(j) for i in range(1, number_of_days+1) for j in range(1, number_of_days+1)]
variable_names
dv_variables = pulp.LpVariable.matrix("x", variable_names, cat = 'Integer', lowBound = 0, upBound=1)
allocation = np.array(dv_variables).reshape(number_of_days, number_of_days)
print('The example of decision variables')
display(allocation[:5][:5])

The example of decision variables


array([[x_11, x_12, x_13, x_14, x_15],
       [x_21, x_22, x_23, x_24, x_25],
       [x_31, x_32, x_33, x_34, x_35],
       [x_41, x_42, x_43, x_44, x_45],
       [x_51, x_52, x_53, x_54, x_55]], dtype=object)

In [ ]:
# initial model
model = pulp.LpProblem("ATM Replenishment", pulp.LpMinimize)

# add objective function
obj_func = pulp.lpSum(allocation * total_costs_arr)
model += obj_func

# add constrints
model += pulp.lpSum(allocation[0]) == 1 # need first day loading date
model += pulp.lpSum(allocation[i][-1] for i in range(number_of_days)) == 1 # make sure load until last day
# Assigning the next trigger date based on the first replenishment date. 
# For example, if the observed period is 5 days and the first schedule is X13 then force the next order to start with X4{4|5}.
for i in range(number_of_days):
    for j in range(number_of_days):
        if i == 0 and j == 0:
            const = pulp.lpSum(allocation[i+1][j+1:]) == pulp.lpSum(allocation[i][j])
            model += const
        elif i == j:
            const = pulp.lpSum(allocation[i+1][j+1:]) == pulp.lpSum([e[j] for e in allocation[:i+1]])
            model += const
        if i > number_of_days - 2:
            break

print(model)

In [ ]:
# Solve model
model.solve()

# Display results
print(f"Model status : {pulp.LpStatus[model.status]}")
print(f"Total cost: {model.objective.value()}")
for v in model.variables():
    try:
        if v.value() == 1:
            print(f"{v.name} = {v.value()}")
    except:
        print('error could not found value')

Model status : Optimal
Total cost: 15.01
x_13 = 1.0
x_45 = 1.0


# Reference

- [1] Özer, Fazilet & Toroslu, Ismail & KARAGOZ, Pinar. (2020). [Comparison of Integer Linear Programming and Dynamic Programming Approaches for ATM Cash Replenishment Optimization Problem.](https://www.igi-global.com/gateway/article/251842) International Journal of Applied Metaheuristic Computing. 11. 120-132. 10.4018/IJAMC.2020070107.